In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np



ParseError: Failed to construct dataset imdb_reviewsMessage type "tensorflow_datasets.DatasetInfo" has no field named "configDescription".
 Available Fields(except extensions): ['name', 'description', 'version', 'citation', 'sizeInBytes', 'downloadSize', 'location', 'downloadChecksums', 'schema', 'splits', 'supervisedKeys', 'redistributionInfo']

In [3]:
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

train_data,test_dfata=imdb['train'],imdb['test']

train_sentences=[]
train_labels=[]

test_sentences=[]
test_labels=[]

for s,l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())

for s,l in test_dfata:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

ParseError: Failed to construct dataset imdb_reviewsMessage type "tensorflow_datasets.DatasetInfo" has no field named "configDescription".
 Available Fields(except extensions): ['name', 'description', 'version', 'citation', 'sizeInBytes', 'downloadSize', 'location', 'downloadChecksums', 'schema', 'splits', 'supervisedKeys', 'redistributionInfo']

In [ ]:
OOV_str='<OOV>'
dim=3
vocab_size=10000
max_len=120
trunc_type='post'


In [ ]:

from  tensorflow.keras.preprocessing.text import Tokenizer
from  tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=OOV_str)
tokenizer.fit_on_texts(train_sentences)
seq=tokenizer.texts_to_sequences(train_sentences)
word_index = tokenizer.word_index


In [6]:
padded=pad_sequences(seq,maxlen=max_len,truncating=trunc_type)


test_seq=tokenizer.texts_to_sequences(test_sentences)
test_pad=pad_sequences(test_seq,maxlen=max_len,truncating=trunc_type)

In [7]:
model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()
#model.fit(x=padded,y=train_labels,batch_size=32,epochs=10,validation_data=(test_pad,test_labels))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 3)            30000     
_________________________________________________________________
flatten (Flatten)            (None, 360)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 2166      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 32,173
Trainable params: 32,173
Non-trainable params: 0
_________________________________________________________________


In [9]:
model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
                           tf.keras.layers.Conv1D(128,5,activation='relu'),
                           tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dense(units=6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 3)            30000     
_________________________________________________________________
conv1d (Conv1D)              (None, 116, 128)          2048      
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 32,829
Trainable params: 32,829
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
#                           tf.keras.layers.Flatten(),
#                           tf.keras.layers.Dense(units=6,activation='relu'),
#                           tf.keras.layers.Dense(1,activation='sigmoid')])

#model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
#                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#                           tf.keras.layers.Dense(units=6,activation='relu'),
#                           tf.keras.layers.Dense(1,activation='sigmoid')])

#model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
#                           tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
#                           tf.keras.layers.Dense(units=6,activation='relu'),
#                           tf.keras.layers.Dense(1,activation='sigmoid')])

#model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,dim,input_length=max_len),
#                           tf.keras.layers.Conv1D(128,5,activation='relu'),
#                           tf.keras.layers.GlobalAveragePooling1D(),
#                           tf.keras.layers.Dense(units=6,activation='relu'),
#                           tf.keras.layers.Dense(1,activation='sigmoid')])

In [13]:

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 3)


In [14]:
import io
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [16]:
reverse_word_index

{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it',
 11: 'i',
 12: 'this',
 13: 'that',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: "'s",
 22: 'on',
 23: 'you',
 24: 'not',
 25: 'are',
 26: 'his',
 27: 'he',
 28: 'have',
 29: 'be',
 30: 'one',
 31: 'all',
 32: 'at',
 33: 'by',
 34: 'they',
 35: 'an',
 36: 'who',
 37: 'so',
 38: 'from',
 39: 'like',
 40: 'her',
 41: "'t",
 42: 'or',
 43: 'just',
 44: 'there',
 45: 'about',
 46: 'out',
 47: "'",
 48: 'has',
 49: 'if',
 50: 'some',
 51: 'what',
 52: 'good',
 53: 'more',
 54: 'very',
 55: 'when',
 56: 'she',
 57: 'up',
 58: 'can',
 59: 'b',
 60: 'time',
 61: 'no',
 62: 'even',
 63: 'my',
 64: 'would',
 65: 'which',
 66: 'story',
 67: 'only',
 68: 'really',
 69: 'see',
 70: 'their',
 71: 'had',
 72: 'were',
 73: 'me',
 74: 'well',
 75: 'we',
 76: 'than',
 77: 'much',
 78: 'been',
 79: 'get',
 80: 'bad',
 81: 'will',
 82: 'people',
 83: 'do',

In [18]:
reverse_word_index [1]

'<OOV>'

In [19]:
weights[0]

array([0.02590623, 0.04111373, 0.03918682], dtype=float32)